In [53]:
# Import LUSID
import lusid.models as models
import lusid
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.5.2521.0


## 1) Create your Instrument Universe

Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here LUSID Knowledge Base: What is an Instrument?.

As part of this definition you will also attach two alias identifiers to your instruments. Read more about alias identifiers here LUSID Knowledge Base: What is an Alias Identifier?.

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here LUSID Knowledge Base: Upsert.

For further usage of the upsert instruments API call refer to the LUSID API Docs: Upserting Instruments.

Run the cell below to upsert your instruments into LUSID

In [2]:
def upsert_instruments(instrument_universe=None, identifier_columns=None):
    """
    This function upserts instruments from a dataframe 
    
    Keyword arguments:
    instrument_universe (Pandas DataFrame) -- The imported instrument universe
    identifer_columns (Tuple (string, string) -- The identifier name in CSV file 
    and identifier name in LUSID
    
    Returns:
    N/A
    """
    
    # Initialise your batch upsert request
    batch_upsert_request = {}

    # Using your instrument universe create your batch request
    for index, instrument in instrument_universe.iterrows():
    
        # Create your identifiers
        identifiers = {}
        for identifier in identifier_columns:
            identifiers[identifier[1]] = models.InstrumentIdValue(
                value=instrument[identifier[0]])

        # Build your request and add it to the dictionary
        batch_upsert_request[instrument['InstrumentName']] = models.InstrumentDefinition(
            name=instrument['InstrumentName'],
            identifiers=identifiers
        )
        
    # Call LUSID to upsert your instrument defintions
    instrument_response = client.instruments.upsert_instruments(
        request_body=batch_upsert_request)
    
    # Pretty print the response
    prettyprint.instrument_response(instrument_response)

In [3]:
equity_source = pd.read_csv('data/multiplesystems-instruments-equities.csv')
equity_source.head()

InstrumentName ClientInternal Currency          Isin  \
0                Amazon_Nasdaq_AMZN   imd_34634534      USD  US0231351067   
1                 Apple_Nasdaq_AAPL   imd_35345345      USD  US0378331005   
2               BP_LondonStockEx_BP   imd_43535553      GBP  GB0007980591   
3  BurfordCapital_LondonStockEx_BUR   imd_43534356      GBP  GG00B4L84979   
4  EKFDiagnostics_LondonStockEx_EKF   imd_34535355      GBP  GB0031509804   

           Figi ExchangeCode           CountryIssue Ticker MarketSector  \
0  BBG000BVPXP1           UN  united_states_america   AMZN       equity   
1  BBG000B9XVV8           UN  united_states_america   AAPL       equity   
2  BBG000C05BD1           LN         united_kingdom    BP/       equity   
3  BBG000PN88Q7           LN         united_kingdom    BUR       equity   
4  BBG000BVNBN3           LN         united_kingdom    EKF       equity   

   SecurityType  Coupon  
0  common_stock     NaN  
1  common_stock     NaN  
2  common_stock     NaN  
3  common_stock     NaN  
4  common_stock     NaN

In [4]:
upsert_instruments(
    instrument_universe=equity_source, 
    identifier_columns=[
        ('ClientInternal', 'ClientInternal'),
        ('Isin', 'Isin'),
        ('Figi', 'Figi'),
        ('Ticker', 'Ticker')
    ])

Instrument Successfully Upserted: BurfordCapital_LondonStockEx_BUR
ClientInternal ID: imd_43534356
LUSID Instrument ID: LUID_P1739DDC


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
ClientInternal ID: imd_35345345
LUSID Instrument ID: LUID_HS1DA84K


Instrument Successfully Upserted: BP_LondonStockEx_BP
ClientInternal ID: imd_43535553
LUSID Instrument ID: LUID_LDYURLNU


Instrument Successfully Upserted: Glencore_LondonStockEx_GLEN
ClientInternal ID: imd_34534555
LUSID Instrument ID: LUID_S6N347I3


Instrument Successfully Upserted: Kingfisher_LondonStockEx_KGF
ClientInternal ID: imd_34535552
LUSID Instrument ID: LUID_KHHWEILW


Instrument Successfully Upserted: WPP_LondonStockEx_WPP
ClientInternal ID: imd_34536734
LUSID Instrument ID: LUID_46W54EOB


Instrument Successfully Upserted: Whitebread_LondonStockEx_WTB
ClientInternal ID: imd_35349900
LUSID Instrument ID: LUID_7RXE0ELE


Instrument Successfully Upserted: JustEat_LondonStockEx_JE
ClientInternal ID: imd_35436366
LUSID Ins

In [5]:
bond_source = pd.read_csv('data/multiplesystems-instruments-bonds.csv')
bond_source.head()

InstrumentName ClientInternal Currency          Isin  Figi  \
0   UKGiltTreasury_2.0_2025   imd_34534536      GBP  GB00BTHH2R79   NaN   
1   UKGiltTreasury_3.5_2045   imd_54234532      GBP  GB00BN65R313   NaN   
2  UKGiltTreasury_3.75_2021   imd_34643653      GBP  GB00B4RMG977   NaN   
3   UKGiltTreasury_4.5_2034   imd_34534534      GBP  GB00B52WS153   NaN   
4      USTreasury_2.00_2021   imd_34535347      USD  US912828U816   NaN   

  ExchangeCode           CountryIssue             Ticker MarketSector  \
0           LN         united_kingdom     UKT 2 09/07/25         govt   
1           LN         united_kingdom   UKT 3.5 01/22/45         govt   
2           LN         united_kingdom  UKT 3.75 09/07/21         govt   
3           LN         united_kingdom   UKT 4.5 09/07/34         govt   
4       BERLIN  united_states_america       T 2 12/31/21         govt   

    SecurityType  Coupon  
0  uk_gilt_stock    2.00  
1  uk_gilt_stock    3.50  
2  uk_gilt_stock    3.75  
3  uk_gilt_stock    4.50  
4  us_government    2.00

In [6]:
upsert_instruments(
    instrument_universe=bond_source, 
    identifier_columns=[
        ('ClientInternal', 'ClientInternal'),
        ('Isin', 'Isin'),
        ('Ticker', 'Ticker')
    ])

Instrument Successfully Upserted: UKGiltTreasury_2.0_2025
ClientInternal ID: imd_34534536
LUSID Instrument ID: LUID_84FPLOHP


Instrument Successfully Upserted: UKGiltTreasury_3.5_2045
ClientInternal ID: imd_54234532
LUSID Instrument ID: LUID_G54PWWXO


Instrument Successfully Upserted: USTreasury_2.00_2021
ClientInternal ID: imd_34535347
LUSID Instrument ID: LUID_CZ7TP760


Instrument Successfully Upserted: USTreasury_6.875_2025
ClientInternal ID: imd_34534539
LUSID Instrument ID: LUID_UVIRET49


Instrument Successfully Upserted: UKGiltTreasury_3.75_2021
ClientInternal ID: imd_34643653
LUSID Instrument ID: LUID_Z35EE199


Instrument Successfully Upserted: UKGiltTreasury_4.5_2034
ClientInternal ID: imd_34534534
LUSID Instrument ID: LUID_3ZHAZR8Q


6  instruments upserted successfully
0  instrument upsert failures


In [7]:
option_source = pd.read_csv('data/multiplesystems-instruments-options.csv')
option_source.head()

InstrumentName ClientInternal Currency  Isin          Figi  \
0   October 19 Calls on AMZN US   imd_84634539      USD   NaN  BBG00NFXK409   
1    October 19 Puts on AAPL US   imd_85345347      USD   NaN  BBG00NBRV912   
2  September 19 Calls on BP/ LN   imd_83535553      GBP   NaN  BBG00M2Z8958   

  FigiComposite  Sedol ExchangeCode           CountryIssue  \
0  BBG00NFXK409    NaN           UN  united_states_america   
1  BBG00NBRV912    NaN           UN  united_states_america   
2           NaN    NaN           LN         united_kingdom   

                Ticker MarketSector   SecurityType  Coupon  
0  AMZN 10/18/19 C1365       equity  equity_option     NaN  
1   AAPL 10/18/19 P140       equity  equity_option     NaN  
2    BPA 09/20/19 C570       equity  equity_option     NaN

In [8]:
upsert_instruments(
    instrument_universe=option_source, 
    identifier_columns=[
        ('ClientInternal', 'ClientInternal'),
        ('Figi', 'Figi'),
        ('Ticker', 'Ticker')
    ])

Instrument Successfully Upserted: October 19 Puts on AAPL US
ClientInternal ID: imd_85345347
LUSID Instrument ID: LUID_9U1W78BB


Instrument Successfully Upserted: September 19 Calls on BP/ LN
ClientInternal ID: imd_83535553
LUSID Instrument ID: LUID_FR4JL070


Instrument Successfully Upserted: October 19 Calls on AMZN US
ClientInternal ID: imd_84634539
LUSID Instrument ID: LUID_HTNT5VFB


3  instruments upserted successfully
0  instrument upsert failures


## 2) Set up a Scope for each Source System

In [9]:
bonds_scope = 'bonds_system_{}'.format(str(uuid.uuid4())[:4])
equities_scope = 'equities_system_{}'.format(str(uuid.uuid4())[:4])
options_scope = 'options_system_{}'.format(str(uuid.uuid4())[:4])

scopes = [bonds_scope, equities_scope, options_scope]

prettyprint.heading('Bonds Scope', bonds_scope)
prettyprint.heading('Equities Scope', bonds_scope)
prettyprint.heading('Options Scope', bonds_scope)

Bonds Scope: bonds_system_3131
Equities Scope: bonds_system_3131
Options Scope: bonds_system_3131


## 3) Create your Portfolio in Each Scope

In [10]:
# Set the code of your portfolio
portfolio_code = 'Global-Strategies'

# Set the creation date of your portfolio 
portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name='Global Strategies Fund',
    code=portfolio_code,
    base_currency='USD',
    description=None,
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=None,
    properties=None)


for scope in scopes:
    
    # Call LUSID to create your portfolio
    response = client.transaction_portfolios.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=request)

    # Pretty print the response
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: bonds_system_3131
Code: Global-Strategies
Portfolio Effective From: 2016-05-30 15:48:02.372179+00:00
Portfolio Created On: 2019-04-17 15:48:01.293393+00:00

Portfolio Created
Scope: equities_system_5382
Code: Global-Strategies
Portfolio Effective From: 2016-05-30 15:48:02.372179+00:00
Portfolio Created On: 2019-04-17 15:48:01.571507+00:00

Portfolio Created
Scope: options_system_024b
Code: Global-Strategies
Portfolio Effective From: 2016-05-30 15:48:02.372179+00:00
Portfolio Created On: 2019-04-17 15:48:01.798191+00:00



## 3) Set your Initial Holdings

In [11]:
def load_holdings(holdings, scope, code, holdings_effective_date, instrument_identifier):
    """
    This function....
    
    Keyword arguments:
    
    
    Returns:
    """
    
    # Iterate the portfolios in the holdings CSV, note in this case you only have one
    for portfolio in holdings['PortfolioCode'].unique():
        # Initialise a list to hold your adjustments
        holding_adjustments = []

        # Iterate over the holdings in each portfolio
        for index, holding in holdings.loc[
            holdings['PortfolioCode'] == portfolio].iterrows():

            # Set your instrument identifiers based on whether or not instrument is cash
            if 'Cash' in holding['InstrumentName']:
                identifier_key = 'Instrument/default/Currency'
                identifier = holding['InstrumentName'].split('_')[0]
            else:
                identifier_key = 'Instrument/default/{}'.format(instrument_identifier)
                identifier = holding[instrument_identifier]

            # Create your holding adjustment and append it to your list
            holding_adjustments.append(
                models.AdjustHoldingRequest(
                    instrument_identifiers={
                        identifier_key: identifier},
                    tax_lots=[
                        models.TargetTaxLotRequest(
                            units=holding['Quantity'],
                            cost=models.CurrencyAndAmount(
                                amount=holding['Quantity'] * holding['Price'],
                                currency=holding['Currency']),
                            portfolio_cost=holding['Quantity'] * holding['Price'],
                            price=holding['Price'])
                    ]
                )
            )

        # Call LUSID to set your initial holdings
        response = client.transaction_portfolios.set_holdings(
            scope=scope,
            code=code,
            effective_at=holdings_effective_date,
            adjust_holding_request=holding_adjustments)

        # Pretty print our response from LUSID
        prettyprint.set_holdings_response(response, scope, portfolio)

In [12]:
holdings = pd.read_csv('data/multiplesystems-holdings-bonds.csv')
holdings.head()

PortfolioCode            InstrumentName  Quantity    Price Currency  \
0  Global-Strategies   UKGiltTreasury_2.0_2025    405589  106.637      GBP   
1  Global-Strategies   UKGiltTreasury_3.5_2045    266169  134.433      GBP   
2  Global-Strategies  UKGiltTreasury_3.75_2021    661713  108.126      GBP   
3  Global-Strategies   UKGiltTreasury_4.5_2034     77481  140.572      GBP   
4  Global-Strategies      USTreasury_2.00_2021   1440244   97.900      USD   

   Figi ClientInternal  
0   NaN   imd_34534536  
1   NaN   imd_54234532  
2   NaN   imd_34643653  
3   NaN   imd_34534534  
4   NaN   imd_34535347

In [13]:
# Make the holdings effective from two days ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=2)

load_holdings(
    holdings=holdings, 
    scope=bonds_scope, 
    code=portfolio_code, 
    holdings_effective_date=holdings_effective_date, 
    instrument_identifier='ClientInternal')

Holdings Successfully Set for Portfolio
Scope: bonds_system_3131
Code: Global-Strategies
Holdings Effective From: 2019-04-15 15:48:03.346331+00:00
Holdings Created On: 2019-04-17 15:48:03.808596+00:00



In [14]:
holdings = pd.read_csv('data/multiplesystems-holdings-equities.csv')
holdings.head()

PortfolioCode                    InstrumentName  Quantity   Price  \
0  Global-Strategies                          GBP_Cash   5557333   1.000   
1  Global-Strategies       Glencore_LondonStockEx_GLEN    905141   2.762   
2  Global-Strategies      Kingfisher_LondonStockEx_KGF   1362038   2.276   
3  Global-Strategies  BurfordCapital_LondonStockEx_BUR    853486  14.060   
4  Global-Strategies  EKFDiagnostics_LondonStockEx_EKF    925925   0.270   

  Currency          Figi  
0      GBP           NaN  
1      GBP  BBG001MM1KV4  
2      GBP  BBG000BKH1W6  
3      GBP  BBG000PN88Q7  
4      GBP  BBG000BVNBN3

In [15]:
load_holdings(
    holdings=holdings, 
    scope=equities_scope, 
    code=portfolio_code, 
    holdings_effective_date=holdings_effective_date, 
    instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: equities_system_5382
Code: Global-Strategies
Holdings Effective From: 2019-04-15 15:48:03.346331+00:00
Holdings Created On: 2019-04-17 15:48:05.613581+00:00



In [16]:
holdings = pd.read_csv('data/multiplesystems-holdings-options.csv')
holdings.head()

PortfolioCode                InstrumentName  Quantity  Price Currency  \
0  Global-Strategies   October 19 Calls on AMZN US       100  24528      USD   
1  Global-Strategies    October 19 Puts on AAPL US       150    383      USD   
2  Global-Strategies  September 19 Calls on BP/ LN       124    140      GBP   

           Figi  
0  BBG00NFXK409  
1  BBG00NBRV912  
2  BBG00M2Z8958

In [17]:
load_holdings(
    holdings=holdings, 
    scope=options_scope, 
    code=portfolio_code, 
    holdings_effective_date=holdings_effective_date, 
    instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: options_system_024b
Code: Global-Strategies
Holdings Effective From: 2019-04-15 15:48:03.346331+00:00
Holdings Created On: 2019-04-17 15:48:06.694829+00:00



## 4) Load your Transactions

In [18]:
def load_transactions(
    transactions, 
    scope, 
    code, 
    trade_date, 
    settlement_date, 
    instrument_identifier,
    source_system):
    """
    This function....
    
    Keyword arguments:
 
    
    Returns:
    """
    
    # Iterate the portfolios in the holdings CSV, note in this case you only have one
    for portfolio in holdings['PortfolioCode'].unique():
        # Initialise a list to hold your adjustments
        transactions_requests = []

        # Iterate over the holdings in each portfolio
        for index, transaction in transactions.loc[
            transactions['PortfolioCode'] == portfolio].iterrows():

            # Set your instrument identifiers based on whether or not instrument is cash
            if 'Cash' in transaction['InstrumentName']:
                identifier_key = 'Instrument/default/Currency'
                identifier = transaction['InstrumentName'].split('_')[0]
            else:
                identifier_key = 'Instrument/default/{}'.format(instrument_identifier)
                identifier = transaction[instrument_identifier]
            
            if transaction['Quantity'] == transaction['SettlementAmt']:
                price = 1
            else:
                price = transaction['Price']
            
            # Create your holding adjustment and append it to your list
            transactions_requests.append(
                models.TransactionRequest(
                    transaction_id=transaction['TransactionId'],
                    type=transaction['TransactionType'],
                    instrument_identifiers={
                        identifier_key: identifier},
                    transaction_date=trade_date,
                    settlement_date=settlement_date,
                    units=transaction['Quantity'],
                    transaction_price=models.TransactionPrice(
                        price=price,
                        type='Price'),
                    total_consideration=models.CurrencyAndAmount(
                        amount=transaction['SettlementAmt'],
                        currency=transaction['SettlementCurrency']),
                    source=source_system,
                    transaction_currency=transaction['TransactionCurrency']))

        # Call LUSID to set your initial holdings
        response = client.transaction_portfolios.upsert_transactions(
            scope=scope,
            code=code,
            transaction_request=transactions_requests)

        # Pretty print our response from LUSID
        prettyprint.transactions_response(response, scope, code)

In [19]:
trade_date = datetime.now(pytz.UTC)
settlement_date = datetime.now(pytz.UTC) + timedelta(days=2)

In [20]:
transactions = pd.read_csv('data/multiplesystems-transactions-bonds.csv')
transactions.head()

PortfolioCode            InstrumentName     Quantity  Price Currency  \
0  Global-Strategies   UKGiltTreasury_2.0_2025  2222.410000    NaN      GBP   
1  Global-Strategies   UKGiltTreasury_3.5_2045  2552.305479    NaN      GBP   
2  Global-Strategies  UKGiltTreasury_3.75_2021  6798.421233    NaN      GBP   
3  Global-Strategies   UKGiltTreasury_4.5_2034   955.245205    NaN      GBP   
4  Global-Strategies      USTreasury_2.00_2021  7891.747945    NaN      USD   

   Figi ClientInternal TransactionType  TransactionId  TransactionDate  \
0   NaN   imd_34534536            ACCR        1852034              NaN   
1   NaN   imd_54234532            ACCR        1852047              NaN   
2   NaN   imd_34643653            ACCR        1852071              NaN   
3   NaN   imd_34534534            ACCR        1852078              NaN   
4   NaN   imd_34535347            ACCR        1852126              NaN   

   SettlementDate  SettlementAmt  TransactionToSettlementExRate  \
0             NaN    2222.410000                              1   
1             NaN    2552.305479                              1   
2             NaN    6798.421233                              1   
3             NaN     955.245205                              1   
4             NaN    7891.747945                              1   

  TransactionCurrency SettlementCurrency  
0                 GBP                GBP  
1                 GBP                GBP  
2                 GBP                GBP  
3                 GBP                GBP  
4                 USD                USD

In [21]:
load_transactions(
    transactions, 
    bonds_scope, 
    portfolio_code, 
    trade_date, 
    settlement_date, 
    'ClientInternal',
    'Bonds')

Transactions Successfully Upserted into Portfolio
Scope: bonds_system_3131
Code: Global-Strategies
Transactions Effective From: 2019-04-17 15:48:08.063873+00:00
Transactions Created On: 2019-04-17 15:48:08.620063+00:00



In [22]:
transactions = pd.read_csv('data/multiplesystems-transactions-equities.csv')
transactions.head()

PortfolioCode                    InstrumentName     Quantity   Price  \
0  Global-Strategies       Glencore_LondonStockEx_GLEN  249999.9442     NaN   
1  Global-Strategies      Kingfisher_LondonStockEx_KGF  125000.0000   2.345   
2  Global-Strategies  BurfordCapital_LondonStockEx_BUR   32050.0000  14.020   
3  Global-Strategies  EKFDiagnostics_LondonStockEx_EKF   31504.0000   0.260   
4  Global-Strategies          JustEat_LondonStockEx_JE  250000.0000   5.478   

  Currency          Figi  ClientInternal TransactionType  TransactionId  \
0      GBP  BBG001MM1KV4             NaN             DIV        5788392   
1      GBP  BBG000BKH1W6             NaN             BUY        5788411   
2      GBP  BBG000PN88Q7             NaN       SHORTSELL        5788429   
3      GBP  BBG000BVNBN3             NaN            SELL        5788497   
4      GBP  BBG0065YWM39             NaN             BUY        5788522   

   TransactionDate  SettlementDate  SettlementAmt  \
0              NaN             NaN   2.499999e+05   
1              NaN             NaN   2.931250e+05   
2              NaN             NaN   4.493410e+05   
3              NaN             NaN   8.191040e+03   
4              NaN             NaN   1.369500e+06   

   TransactionToSettlementExRate TransactionCurrency SettlementCurrency  
0                              1                 GBP                GBP  
1                              1                 GBP                GBP  
2                              1                 GBP                GBP  
3                              1                 GBP                GBP  
4                              1                 GBP                GBP

In [23]:
load_transactions(
    transactions, 
    equities_scope, 
    portfolio_code, 
    trade_date, 
    settlement_date, 
    'Figi',
    'Equities')

Transactions Successfully Upserted into Portfolio
Scope: equities_system_5382
Code: Global-Strategies
Transactions Effective From: 2019-04-17 15:48:08.063873+00:00
Transactions Created On: 2019-04-17 15:48:10.271479+00:00



In [24]:
transactions = pd.read_csv('data/multiplesystems-transactions-options.csv')
transactions.head()

PortfolioCode               InstrumentName  Quantity   Price Currency  \
0  Global-Strategies  October 19 Calls on AMZN US         6  243.25      USD   

           Figi  ClientInternal TransactionType  TransactionId  \
0  BBG00NFXK409             NaN    CALLCONTRACT       98034324   

   TransactionDate  SettlementDate  SettlementAmt  \
0              NaN             NaN         1459.5   

   TransactionToSettlementExRate TransactionCurrency SettlementCurrency  
0                              1                 USD                USD

In [25]:
load_transactions(
    transactions, 
    options_scope, 
    portfolio_code, 
    trade_date, 
    settlement_date, 
    'Figi',
    'Options')

Transactions Successfully Upserted into Portfolio
Scope: options_system_024b
Code: Global-Strategies
Transactions Effective From: 2019-04-17 15:48:08.063873+00:00
Transactions Created On: 2019-04-17 15:48:11.852223+00:00



## 5) Get your Holdings

In [26]:
# Try and get your holdings, otherwise provide the error message
try:
    response = client.transaction_portfolios.get_holdings(
        scope=bonds_scope,
        code=portfolio_code,
        instrument_property_keys=['Instrument/default/Name'])
    prettyprint.holdings_response(response, scope, portfolio_code)
except lusid.rest.ApiException as e:
    print ('Error retrieving holdings as {}'.format(e))

Holdings for Portfolio
Scope: options_system_024b
Code: Global-Strategies

Instrument Name: UKGiltTreasury_2.0_2025
Units: 405589.0
Cost: 43250794.19
Currency: GBP


Instrument Name: UKGiltTreasury_2.0_2025
Units: -15000.0
Cost: -1835700.0
Currency: USD


Instrument Name: UKGiltTreasury_3.5_2045
Units: 266169.0
Cost: 35781897.18
Currency: GBP


Instrument Name: UKGiltTreasury_3.75_2021
Units: 618713.0
Cost: 66898961.84
Currency: GBP


Instrument Name: UKGiltTreasury_4.5_2034
Units: 97481.0
Cost: 13748259.13
Currency: GBP


Instrument Name: USTreasury_2.00_2021
Units: 1440244.0
Cost: 140999887.6
Currency: USD


Instrument Name: USTreasury_6.875_2025
Units: 534049.0
Cost: 66499781.48
Currency: USD


Instrument Name: USD
Units: 23000000.0
Cost: 23000000.0
Currency: USD


Instrument Name: GBP
Units: 2778667.0
Cost: 2778667.0
Currency: GBP


Instrument Name: GBP
Units: 2222.41
Cost: 2222.41
Currency: GBP
Unsettled Transaction Id: 1852034
Settlement Date: 2019-04-19 15:48:08.063935+00:00


I

## 6) Update your Transaction Type Configuration

In [27]:
# Import the default transaction type configuration and load it into a dictionary 
default_transaction_mapping=open('data/default_transaction_mapping.json').read()
default_transaction_mapping = json.loads(default_transaction_mapping)
# Pretty print your configuration
pprint.pprint(default_transaction_mapping)

{'values': [{'aliases': [{'description': 'Purchase',
                          'transactionClass': 'Basic',
                          'transactionGroup': 'default',
                          'transactionRoles': 'LongLonger',
                          'type': 'Buy'},
                         {'description': 'PURCHASE',
                          'transactionClass': 'Basic',
                          'transactionGroup': 'alt1',
                          'transactionRoles': 'LongLonger',
                          'type': 'BY'}],
             'movements': [{'direction': 1,
                            'mappings': [],
                            'movementTypes': 'StockMovement',
                            'properties': [],
                            'side': 'Side1'},
                           {'direction': -1,
                            'mappings': [],
                            'movementTypes': 'CashCommitment',
                            'properties': [],
                            '

In [28]:
# Initialise your list of configuration requests, one for each transaction type
configuration_requests = []

# Iterate over your configurations in the default mapping
for configuration in default_transaction_mapping['values']:
    
    # Initialise your list of aliases for this configuration
    aliases = []
    
    # Iterate over the aliases in the imported config 
    for alias in configuration['aliases']:
        # Append the alias to your list
        aliases.append(
            models.TransactionConfigurationTypeAlias(
                type=alias['type'],
                description=alias['description'],
                transaction_class=alias['transactionClass'],
                transaction_group=alias['transactionGroup'],
                transaction_roles=alias['transactionRoles']))
        
        
    # Initialise your list of movements for this configuration
    movements = []
    
    # Iterate over the movements in the impoted config
    for movement in configuration['movements']:
        
        # Add properties if they exist in the config
        if len (movement['properties']) > 0:
            key = movement['properties'][0]['key']
            value = models.PerpetualPropertyValue(movement['properties'][0]['value'])
            properties={key: value}                                    
        else:
            properties=None
            
        # Append the movement to your list
        movements.append(
            models.TransactionConfigurationMovementDataRequest(
                movement_types=movement['movementTypes'],
                side=movement['side'],
                direction=movement['direction'],
                properties=properties,
                mappings=None))
    
    # Build your configuration for this transaction type
    configuration_requests.append(
        models.TransactionConfigurationDataRequest(
        aliases=aliases,
        movements=movements,
        properties=None))

# Call LUSID to set your configuration for our transaction types
response = client.system_configuration.set_configuration_transaction_types(
    transaction_configuration_data_request=configuration_requests)

# Pretty print the response
prettyprint.transaction_type_response(response)

Transaction Configuration #1

Transaction Type Aliases
Transaction Type: Buy
Alias Description: Purchase
Transaction Class: Basic
Transaction Group: default
Transaction Roles: LongLonger


Transaction Type: BY
Alias Description: PURCHASE
Transaction Class: Basic
Transaction Group: alt1
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #2

Transaction Type Aliases
Transaction Type: Sell
Alias Description: Sale
Transaction Class: Basic
Transaction Group: default
Transaction Roles: LongShorter


Transaction Type: SL
Alias Description: SALE
Transaction Class: Basic
Transaction Group: alt1
Transaction Roles: LongShorter


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #3

Transaction Type Aliases
Transaction Type: CoverShort
Alias Des

In [29]:
transaction_types = pd.read_csv(
    'data/multiplesystems-transactions-bonds.csv')['TransactionType'].unique()
print (transaction_types)

['ACCR' 'PURCHASE' 'SALE']


In [30]:
response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='ACCR',
                description='Interest accrued on a bond',
                transaction_class='BondInstruments',
                transaction_group='Bonds',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['ACCR'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='PURCHASE',
                description='A purchase of a bond',
                transaction_class='BondInstruments',
                transaction_group='Bonds',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['PURCHASE'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='SALE',
                description='The sale of a bond',
                transaction_class='BondInstruments',
                transaction_group='Bonds',
                transaction_roles='LongShorter')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=-1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['SALE'])

Transaction Configuration #17

Transaction Type Aliases
Transaction Type: ACCR
Alias Description: Interest accrued on a bond
Transaction Class: BondInstruments
Transaction Group: Bonds
Transaction Roles: LongLonger


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #18

Transaction Type Aliases
Transaction Type: PURCHASE
Alias Description: A purchase of a bond
Transaction Class: BondInstruments
Transaction Group: Bonds
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #19

Transaction Type Aliases
Transaction Type: SALE
Alias Description: The sale of a bond
Transaction Class: BondInstruments
Transaction Group: Bonds
Transaction Roles: LongShorter


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1





In [31]:
transaction_types = pd.read_csv(
    'data/multiplesystems-transactions-equities.csv')['TransactionType'].unique()
print (transaction_types)

['DIV' 'BUY' 'SHORTSELL' 'SELL']


In [32]:
response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='DIV',
                description='A dividend from an equity',
                transaction_class='EquityInstruments',
                transaction_group='Equities',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['DIV'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='BUY',
                description='The purchase of an equity',
                transaction_class='EquityInstrument',
                transaction_group='Equities',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['BUY'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='SHORTSELL',
                description='A short position created from borrowing and selling an equity',
                transaction_class='EquityInstruments',
                transaction_group='Equities',
                transaction_roles='ShortShorter')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=-1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['SHORTSELL'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='SELL',
                description='A sale of an equity',
                transaction_class='EquityInstruments',
                transaction_group='Equities',
                transaction_roles='LongShorter')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=-1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['SELL'])

Transaction Configuration #20

Transaction Type Aliases
Transaction Type: DIV
Alias Description: A dividend from an equity
Transaction Class: EquityInstruments
Transaction Group: Equities
Transaction Roles: LongLonger


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #21

Transaction Type Aliases
Transaction Type: BUY
Alias Description: The purchase of an equity
Transaction Class: EquityInstrument
Transaction Group: Equities
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #22

Transaction Type Aliases
Transaction Type: SHORTSELL
Alias Description: A short position created from borrowing and selling an equity
Transaction Class: EquityInstruments
Transaction Group: Equities
Transaction Roles: ShortShorter


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: -1
M

In [33]:
transaction_types = pd.read_csv(
    'data/multiplesystems-transactions-options.csv')['TransactionType'].unique()
print (transaction_types)

['CALLCONTRACT']


In [34]:
response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='CALLCONTRACT',
                description='The purchase of a call options contract',
                transaction_class='EquityInstrument',
                transaction_group='Options',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['CALLCONTRACT'])

Transaction Configuration #24

Transaction Type Aliases
Transaction Type: CALLCONTRACT
Alias Description: The purchase of a call options contract
Transaction Class: EquityInstrument
Transaction Group: Options
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1





In [35]:
# Try and get your holdings, otherwise provide the error message
try:
    response = client.transaction_portfolios.get_holdings(
        scope=bonds_scope,
        code=portfolio_code,
        instrument_property_keys=['Instrument/default/Name'])
    prettyprint.holdings_response(response, scope, portfolio_code)
except lusid.rest.ApiException as e:
    print ('Error retrieving holdings as {}'.format(e))

Holdings for Portfolio
Scope: options_system_024b
Code: Global-Strategies

Instrument Name: UKGiltTreasury_2.0_2025
Units: 405589.0
Cost: 43250794.19
Currency: GBP


Instrument Name: UKGiltTreasury_2.0_2025
Units: -15000.0
Cost: -1835700.0
Currency: USD


Instrument Name: UKGiltTreasury_3.5_2045
Units: 266169.0
Cost: 35781897.18
Currency: GBP


Instrument Name: UKGiltTreasury_3.75_2021
Units: 618713.0
Cost: 66898961.84
Currency: GBP


Instrument Name: UKGiltTreasury_4.5_2034
Units: 97481.0
Cost: 13748259.13
Currency: GBP


Instrument Name: USTreasury_2.00_2021
Units: 1440244.0
Cost: 140999887.6
Currency: USD


Instrument Name: USTreasury_6.875_2025
Units: 534049.0
Cost: 66499781.48
Currency: USD


Instrument Name: USD
Units: 23000000.0
Cost: 23000000.0
Currency: USD


Instrument Name: GBP
Units: 2778667.0
Cost: 2778667.0
Currency: GBP


Instrument Name: GBP
Units: 2222.41
Cost: 2222.41
Currency: GBP
Unsettled Transaction Id: 1852034
Settlement Date: 2019-04-19 15:48:08.063935+00:00


I

## 7) Load your End of Day Positions

In [37]:
# Make the holdings effective from two days ago
end_of_day_effective_date = datetime.now(pytz.UTC) - timedelta(days=0.5)

holdings = pd.read_csv('data/multiplesystems-holdings-eod-bonds.csv')
holdings.head()

PortfolioCode            InstrumentName  Quantity    Price Currency  \
0  Global-Strategies   UKGiltTreasury_2.0_2025    405589  106.637      GBP   
1  Global-Strategies   UKGiltTreasury_3.5_2045    266169  134.433      GBP   
2  Global-Strategies  UKGiltTreasury_3.75_2021    618713  108.126      GBP   
3  Global-Strategies   UKGiltTreasury_4.5_2034     97481  140.572      GBP   
4  Global-Strategies      USTreasury_2.00_2021   1440244   97.900      USD   

   Figi ClientInternal  
0   NaN   imd_34534536  
1   NaN   imd_54234532  
2   NaN   imd_34643653  
3   NaN   imd_34534534  
4   NaN   imd_34535347

In [38]:
load_holdings(
    holdings=holdings, 
    scope=bonds_scope, 
    code=portfolio_code, 
    holdings_effective_date=end_of_day_effective_date, 
    instrument_identifier='ClientInternal')

Holdings Successfully Set for Portfolio
Scope: bonds_system_3131
Code: Global-Strategies
Holdings Effective From: 2019-04-17 04:17:15.231727+00:00
Holdings Created On: 2019-04-17 16:18:59.639761+00:00



In [39]:
holdings = pd.read_csv('data/multiplesystems-holdings-eod-equities.csv')
holdings.head()

PortfolioCode                    InstrumentName  Quantity    Price  \
0  Global-Strategies                          GBP_Cash   5557333   1.0000   
1  Global-Strategies       Glencore_LondonStockEx_GLEN    905141   2.7620   
2  Global-Strategies      Kingfisher_LondonStockEx_KGF   1487038   2.2818   
3  Global-Strategies  BurfordCapital_LondonStockEx_BUR    821436  14.0600   
4  Global-Strategies  EKFDiagnostics_LondonStockEx_EKF    894421   0.2700   

  Currency          Figi  
0      GBP           NaN  
1      GBP  BBG001MM1KV4  
2      GBP  BBG000BKH1W6  
3      GBP  BBG000PN88Q7  
4      GBP  BBG000BVNBN3

In [40]:
load_holdings(
    holdings=holdings, 
    scope=equities_scope, 
    code=portfolio_code, 
    holdings_effective_date=end_of_day_effective_date, 
    instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: equities_system_5382
Code: Global-Strategies
Holdings Effective From: 2019-04-17 04:17:15.231727+00:00
Holdings Created On: 2019-04-17 16:19:24.368731+00:00



In [41]:
holdings = pd.read_csv('data/multiplesystems-holdings-options.csv')
holdings.head()

PortfolioCode                InstrumentName  Quantity  Price Currency  \
0  Global-Strategies   October 19 Calls on AMZN US       100  24528      USD   
1  Global-Strategies    October 19 Puts on AAPL US       150    383      USD   
2  Global-Strategies  September 19 Calls on BP/ LN       124    140      GBP   

           Figi  
0  BBG00NFXK409  
1  BBG00NBRV912  
2  BBG00M2Z8958

In [42]:
load_holdings(
    holdings=holdings, 
    scope=options_scope, 
    code=portfolio_code, 
    holdings_effective_date=end_of_day_effective_date, 
    instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: options_system_024b
Code: Global-Strategies
Holdings Effective From: 2019-04-17 04:17:15.231727+00:00
Holdings Created On: 2019-04-17 16:20:43.029077+00:00



## 8) Perform a Bi-Temporal Reconciliation 

In [44]:
# Build your left side of the reconciliation before you made the adjustment
before_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=bonds_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date - timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Build your right side of the reconciliation after you made the adjustment
after_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=bonds_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date + timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Create your reconciliation request
reconcile_holdings_request = models.PortfoliosReconciliationRequest(
    left=before_adjustment,
    right=after_adjustment,
    instrument_property_keys=['Instrument/default/Name', 'Instrument/default/Figi'])

# Reconcile your two portfolios
response = client.reconciliations.reconcile_holdings(
    portfolios_reconciliation_request=reconcile_holdings_request)

# Pretty print the response
prettyprint.reconciliation_response(
    response,
    scope,
    portfolio_code)

Reconciliation Breaks for Portfolio
Scope: options_system_024b
Code: Global-Strategies

Reconciliation Break
Instrument LUID: LUID_Z35EE199
Left Units: 661713.0
Right Units: 618713.0
Difference In Units: -43000.0
Left Cost: 71548379.84
Right Cost: 66898961.84
Difference In Cost: -4649418.0
Currency: GBP


Reconciliation Break
Instrument LUID: LUID_3ZHAZR8Q
Left Units: 77481.0
Right Units: 97481.0
Difference In Units: 20000.0
Left Cost: 10891659.13
Right Cost: 13703099.13
Difference In Cost: 2811440.0
Currency: GBP


Reconciliation Break
Instrument LUID: LUID_UVIRET49
Left Units: 534049.0
Right Units: 519049.0
Difference In Units: -15000.0
Left Cost: 66499781.48
Right Cost: 64631981.48
Difference In Cost: -1867800.0
Currency: USD


{'count': 3,
 'href': 'https://api-am-ci.lusid.com/api/portfolios/$reconcileholdings',
 'links': None,
 'values': [{'difference_cost': {'amount': -4649418.0, 'currency': 'GBP'},
             'difference_units': -43000.0,
             'instrument_properties': 

In [45]:
# Build your left side of the reconciliation before you made the adjustment
before_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=equities_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date - timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Build your right side of the reconciliation after you made the adjustment
after_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=equities_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date + timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Create your reconciliation request
reconcile_holdings_request = models.PortfoliosReconciliationRequest(
    left=before_adjustment,
    right=after_adjustment,
    instrument_property_keys=['Instrument/default/Name', 'Instrument/default/Figi'])

# Reconcile your two portfolios
response = client.reconciliations.reconcile_holdings(
    portfolios_reconciliation_request=reconcile_holdings_request)

# Pretty print the response
prettyprint.reconciliation_response(
    response,
    scope,
    portfolio_code)

Reconciliation Breaks for Portfolio
Scope: options_system_024b
Code: Global-Strategies

Reconciliation Break
Instrument LUID: LUID_KHHWEILW
Left Units: 1362038.0
Right Units: 1487038.0
Difference In Units: 125000.0
Left Cost: 3099998.49
Right Cost: 3393123.49
Difference In Cost: 293125.0
Currency: GBP


Reconciliation Break
Instrument LUID: LUID_P1739DDC
Left Units: 853486.0
Right Units: 821436.0
Difference In Units: -32050.0
Left Cost: 12000013.16
Right Cost: 11549390.16
Difference In Cost: -450623.0
Currency: GBP


Reconciliation Break
Instrument LUID: LUID_IOU8GN2G
Left Units: 925925.0
Right Units: 894421.0
Difference In Units: -31504.0
Left Cost: 249999.75
Right Cost: 241493.67
Difference In Cost: -8506.08
Currency: GBP


Reconciliation Break
Instrument LUID: LUID_KRYXBDZX
Left Units: 4860907.0
Right Units: 5110907.0
Difference In Units: 250000.0
Left Cost: 26559995.85
Right Cost: 27929495.85
Difference In Cost: 1369500.0
Currency: GBP




In [46]:
# Build your left side of the reconciliation before you made the adjustment
before_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=options_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date - timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Build your right side of the reconciliation after you made the adjustment
after_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=options_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date + timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Create your reconciliation request
reconcile_holdings_request = models.PortfoliosReconciliationRequest(
    left=before_adjustment,
    right=after_adjustment,
    instrument_property_keys=['Instrument/default/Name', 'Instrument/default/Figi'])

# Reconcile your two portfolios
response = client.reconciliations.reconcile_holdings(
    portfolios_reconciliation_request=reconcile_holdings_request)

# Pretty print the response
prettyprint.reconciliation_response(
    response,
    scope,
    portfolio_code)

Reconciliation Breaks for Portfolio
Scope: options_system_024b
Code: Global-Strategies

No Reconciliation Breaks


## 9) Group the Sources to Create an Overall View of your Fund

In [49]:
fund_scope = 'combined_systems_{}'.format(str(uuid.uuid4())[:4])

group_request = models.CreatePortfolioGroupRequest(
    id=portfolio_code+'Group',
    display_name=portfolio_code + 'Group')

response = client.portfolio_groups.create_portfolio_group(
    scope=fund_scope,
    create_portfolio_group_request=group_request)

prettyprint.portfolio_group_response(response, 'created')

Portfolio Group Created
Name: Global-StrategiesGroup
Scope: combined_systems_7409
Code: Global-StrategiesGroup
Portfolios Inside Group: 




In [50]:
for scope in scopes:
    
    response = client.portfolio_groups.add_portfolio_to_group(
        scope=fund_scope,
        code=portfolio_code+'Group',
        resource_id=models.ResourceId(
            scope=scope,
            code=portfolio_code))

    prettyprint.get_portfolio_group_response(response)

Portfolio Group: 
Name: Global-StrategiesGroup
Scope: combined_systems_7409
Code: Global-StrategiesGroup
Portfolios Inside Group: 
Global-Strategies
Subgroups Inside Group: 


Portfolio Group: 
Name: Global-StrategiesGroup
Scope: combined_systems_7409
Code: Global-StrategiesGroup
Portfolios Inside Group: 
Global-Strategies
Global-Strategies
Subgroups Inside Group: 


Portfolio Group: 
Name: Global-StrategiesGroup
Scope: combined_systems_7409
Code: Global-StrategiesGroup
Portfolios Inside Group: 
Global-Strategies
Global-Strategies
Global-Strategies
Subgroups Inside Group: 




## 10) Value your Fund

In [51]:
prices = pd.read_csv('data/multiplesystems-prices.csv')
prices.head()

instrument_name currency figi  price_current             ticker  \
0   UKGiltTreasury_2.0_2025      GBP  NaN        106.637     UKT 2 09/07/25   
1   UKGiltTreasury_3.5_2045      GBP  NaN        134.433   UKT 3.5 01/22/45   
2  UKGiltTreasury_3.75_2021      GBP  NaN        108.126  UKT 3.75 09/07/21   
3   UKGiltTreasury_4.5_2034      GBP  NaN        140.572   UKT 4.5 09/07/34   
4      USTreasury_2.00_2021      USD  NaN         97.900       T 2 12/31/21   

           isin  sedol client_internal  
0  GB00BTHH2R79    NaN    imd_34534536  
1  GB00BN65R313    NaN    imd_54234532  
2  GB00B4RMG977    NaN    imd_34643653  
3  GB00B52WS153    NaN    imd_34534534  
4  US912828U816    NaN    imd_34535347

In [82]:
instrument_quotes = []

for index, quote in prices.iterrows():

    if type(quote['figi']) is str:
        instrument_id = quote['figi']
        instrument_id_type = 'Figi'
    elif type(quote['isin']) is str:
        instrument_id = quote['isin']
        instrument_id_type = 'Isin'
    else:
        instrument_id = quote['sedol']
        instrument_id_type = 'Sedol'
    
    instrument_quotes.append(models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            provider='DataScope',
            price_source='USDRC',
            instrument_id=instrument_id,
            instrument_id_type=instrument_id_type,
            quote_type='Price',
            price_side='Mid'),
        metric_value=models.MetricValue(
            value=quote['price_current'],
            unit=quote['currency']),
        effective_at=holdings_effective_date,
        lineage='InternalSystem'
    ))
    
response = client.quotes.upsert_quotes(
    scope=fund_scope,
    upsert_quote_request=instrument_quotes)

print (response)

{'as_at': datetime.datetime(2019, 4, 17, 16, 54, 7, 663189, tzinfo=tzlocal()),
 'links': None}


In [83]:
inline_recipe = models.ConfigurationRecipe(
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.Figi.*',
               supplier='DataScope',
               data_scope=fund_scope,
               quote_type='Price',
               price_side='Mid'),
           models.MarketDataKeyRule(
               key='Equity.Isin.*',
               supplier='DataScope',
               data_scope=fund_scope,
               quote_type='Price',
               price_side='Mid'),
            models.MarketDataKeyRule(
               key='Fx.CurrencyPair.*',
               supplier='DataScope',
               data_scope=fund_scope,
               quote_type='Rate',
               price_side='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='Figi',
            default_scope=scope)
    )
)

aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=end_of_day_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/SubHoldingKey',
        op='Value'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
    ],
    group_by=[
        'Holding/default/SubHoldingKey'
    ])

response = client.aggregation.get_aggregation_by_group(
    scope=fund_scope,
    code=portfolio_code+'Group',
    aggregation_request=aggregation_request)

print (response)

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 17 Apr 2019 16:54:11 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'lusid-meta-success': 'False', 'lusid-meta-duration': '814', 'lusid-meta-requestId': '0HLM389BTO65F:00000001', 'Server': 'FINBOURNE'})
HTTP response body: {"status":404,"code":"PriceNotFound","message":"Failed to find or create market data item [Analytic LUID_84FPLOHP].","detailedMessage":"Failed to find or create market data item [Analytic LUID_84FPLOHP]. This occurred because LUID_84FPLOHP is not in the loaded Analytics set. Implies incorrect dependencies or missing data for time.","items":[],"moreInfo":"https://docs.lusid.com/#section/Error-Codes/189"}


In [77]:
r = client.instruments.get_instrument(identifier_type='LusidInstrumentId',
                                     identifier='LUID_84FPLOHP')

print (r)

{'href': 'https://api-am-ci.lusid.com/api/instruments/LusidInstrumentId/LUID_84FPLOHP',
 'identifiers': {'ClientInternal': 'imd_34534536',
                 'Isin': 'GB00BTHH2R79',
                 'Ticker': 'UKT 2 09/07/25'},
 'instrument_definition': None,
 'links': [{'description': None,
            'href': 'https://api-am-ci.lusid.com/api/schemas/entities/Instrument',
            'method': 'GET',
            'relation': 'EntitySchema'}],
 'lookthrough_portfolio': None,
 'lusid_instrument_id': 'LUID_84FPLOHP',
 'name': 'UKGiltTreasury_2.0_2025',
 'properties': [],
 'state': 'Active',
 'version': {'as_at_date': datetime.datetime(2019, 4, 17, 16, 49, 58, 494785, tzinfo=tzlocal()),
             'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal())}}
